<a href="https://colab.research.google.com/github/panchambanerjee/finetuning_expts/blob/main/synthetic_data_kit_dataset_gen_2025_06_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using **synthetic-data-kit** (https://github.com/meta-llama/synthetic-data-kit/tree/main/use-cases/getting-started) to generate QA pairs for fine-tuning from a Recent interesting Cosmology + ML paper: **Learning and Interpreting Gravitational-Wave Features from CNNs with a Random Forest
Approach** (https://arxiv.org/pdf/2505.20357)

Also using this Unsloth notebook as reference: https://colab.research.google.com/drive/1aRRX5up1XMPR1TBn7lxnk2AHboeZqVG_#scrollTo=jxUolhgCPSr1

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm==0.8.2
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm==0.8.2

# Get https://github.com/meta-llama/synthetic-data-kit
!pip install synthetic-data-kit

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" in "".join(os.environ.keys()):
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers|importlib_metadata)[^\n]{0,}\n", b"", f))
    !pip install -r vllm_requirements.txt

In [ ]:
from unsloth.dataprep import SyntheticDataKit

generator = SyntheticDataKit.from_pretrained(
    # Choose any model from https://huggingface.co/unsloth
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = 2048, # Longer sequence lengths will be slower!
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 06-02 03:24:34 [__init__.py:239] Automatically detected platform cuda.


In [ ]:
generator.prepare_qa_generation(
    output_folder = "data", # Output location of synthetic data
    temperature = 0.7, # Higher temp makes more diverse datases
    top_p = 0.95,
    overlap = 64, # Overlap portion during chunking
    max_generation_tokens = 512, # Can increase for longer QA pairs
)

In [ ]:
!synthetic-data-kit system-check

In [ ]:
### Parse the document to generate QA Pairs

# Byte Latent Transformer: Patches Scale Better Than Tokens paper in HTML format
!synthetic-data-kit \
    -c synthetic_data_kit_config.yaml \
    ingest "https://arxiv.org/pdf/2505.20357"

# Truncate document
filenames = generator.chunk_data("data/output/arxiv_org.txt")
print(len(filenames), filenames[:3])

We see around 2256 chunks of data. We now call synthetic-data-kit to create some pairs of data for 50 of our chunks.



Using `--num-pairs` will generate **approximately** that many QA pairs. However it might be shorter or longer depending on the `max_seq_length` of the loaded up model.

In [7]:
import time
# Process 5 chunks for now -> can increase but slower!
for filename in filenames[:5]:
    !synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        create {filename} \
        --num-pairs 25 \
        --type "qa"
    time.sleep(2) # Sleep some time to leave some room for processing

Processing 1 chunks to generate QA pairs...
Batch processing complete.
Generated 16 QA pairs total
Saving result to data/generated/arxiv_org_0_qa_pairs.json
Successfully wrote test file to data/generated/test_write.json
Successfully wrote result to data/generated/arxiv_org_0_qa_pairs.json
⠹ Generating qa content from data/output/arxiv_org_0.txt...
 Content saved to data/generated/arxiv_org_0_qa_pairs.json
Processing 1 chunks to generate QA pairs...
Batch processing complete.
Generated 15 QA pairs total
Saving result to data/generated/arxiv_org_1_qa_pairs.json
Successfully wrote test file to data/generated/test_write.json
Successfully wrote result to data/generated/arxiv_org_1_qa_pairs.json
⠏ Generating qa content from data/output/arxiv_org_1.txt...
 Content saved to data/generated/arxiv_org_1_qa_pairs.json
Processing 1 chunks to generate QA pairs...
Batch processing complete.
Generated 14 QA pairs total
Saving result to data/generated/arxiv_org_2_qa_pairs.json
Successfully wrote test f

In [8]:
# Optionally, we can clean up the data via pruning "bad" or low quality examples and convert the rest to JSON format for finetuning!

# !synthetic-data-kit \
#     -c synthetic_data_kit_config.yaml \
#     curate --threshold 0.0 \
#     "data/generated/arxiv_org_0_qa_pairs.json"

In [9]:
# Convert the generated datasets into QA formats so we can load it for finetuning

qa_pairs_filenames = [
    f"data/generated/arxiv_org_{i}_qa_pairs.json"
    for i in range(len(filenames[:3]))
]
for filename in qa_pairs_filenames:
    !synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        save-as {filename} -f ft

⠋ Converting data/generated/arxiv_org_0_qa_pairs.json to ft format with json 
storage...
 Converted to ft format and saved to data/final/arxiv_org_0_qa_pairs_ft.json
⠋ Converting data/generated/arxiv_org_1_qa_pairs.json to ft format with json 
storage...
 Converted to ft format and saved to data/final/arxiv_org_1_qa_pairs_ft.json
⠋ Converting data/generated/arxiv_org_2_qa_pairs.json to ft format with json 
storage...
 Converted to ft format and saved to data/final/arxiv_org_2_qa_pairs_ft.json


In [10]:
from datasets import Dataset
import pandas as pd
final_filenames = [
    f"data/final/arxiv_org_{i}_qa_pairs_ft.json"
    for i in range(len(filenames[:3]))
]
conversations = pd.concat([
    pd.read_json(name) for name in final_filenames
]).reset_index(drop = True)

dataset = Dataset.from_pandas(conversations)

In [12]:
len(dataset)

45

In [13]:
final_filenames

['data/final/arxiv_org_0_qa_pairs_ft.json',
 'data/final/arxiv_org_1_qa_pairs_ft.json',
 'data/final/arxiv_org_2_qa_pairs_ft.json']

In [14]:
dataset[0]

{'messages': [{'content': 'You are a helpful assistant.', 'role': 'system'},
  {'content': 'Who are the authors of the PDF document?', 'role': 'user'},
  {'content': 'Jun Tian; He Wang; Jibo He; Yu Pan; Shuo Cao; Qingquan Jiang',
   'role': 'assistant'}]}

In [15]:
dataset[1]

{'messages': [{'content': 'You are a helpful assistant.', 'role': 'system'},
  {'content': 'What is the title of the PDF document?', 'role': 'user'},
  {'content': 'Learning and Interpreting Gravitational-Wave Features from CNNs with a Random Forest Approach',
   'role': 'assistant'}]}

In [16]:
dataset[2]

{'messages': [{'content': 'You are a helpful assistant.', 'role': 'system'},
  {'content': 'What is the DOI of the PDF document?', 'role': 'user'},
  {'content': 'https://doi.org/10.48550/arXiv.2505.20357',
   'role': 'assistant'}]}

In [18]:
dataset[10]

{'messages': [{'content': 'You are a helpful assistant.', 'role': 'system'},
  {'content': 'How many bytes does the metadata object contain?',
   'role': 'user'},
  {'content': '1769', 'role': 'assistant'}]}

In [19]:
dataset[15]

{'messages': [{'content': 'You are a helpful assistant.', 'role': 'system'},
  {'content': 'How many outlines are in the outline object?', 'role': 'user'},
  {'content': '1', 'role': 'assistant'}]}

In [20]:
dataset[40]

{'messages': [{'content': 'You are a helpful assistant.', 'role': 'system'},
  {'content': 'What is the D value of the 26th annot?', 'role': 'user'},
  {'content': '(cite.2019PhRvX...9c1040A)', 'role': 'assistant'}]}

In [21]:
generator.cleanup()

Attempting to terminate the VLLM server gracefully...
Server terminated gracefully.
